In [1]:
import os
import glob
import time
import itertools

from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from scipy.ndimage.measurements import label

from training import *

np.random.seed(0xdeadbeef)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

In [3]:
t=time.time()
cars = glob.iglob('data/vehicles/**/*.png')
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

notcars = glob.iglob('data/non-vehicles/**/*.png')
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

74.19 Seconds to extract features...


In [4]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=np.random.randint(1000))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0xdeadbeef)

In [11]:
clf = Pipeline([('scaling', StandardScaler()),
#                  ('feature_selection', SelectFromModel(ExtraTreesClassifier())),
                ('classification', LinearSVC(loss='hinge')),
               ])

# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

16.23 Seconds to train SVC...


In [12]:
print('Test Accuracy of classifier = ', round(clf.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My classifier predicts: ', clf.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels')

Test Accuracy of classifier =  0.9764
My classifier predicts:  [ 1.  0.  1.  1.  0.  1.  0.  0.  0.  0.]
For these 10 labels:  [ 1.  0.  1.  1.  0.  1.  0.  0.  0.  0.]
0.00765 Seconds to predict 10 labels


In [8]:
## Create hard negative data
# from shutil import copyfile

# def make_hardneg(out_path, neg=True):
#     y_pred = clf.predict(X)

#     if neg:
#         notcars = glob.glob('data/non-vehicles/**/*.png')
#     else:
#         cars = glob.glob('data/vehicles/**/*.png')
#     offset = len(car_features)

#     # Wrong indices
#     wrong = np.nonzero(y_pred != y)[0]
#     for w in wrong:
#         # False positives
#         if neg and y_pred[w] == 1 and y[w] == 0:
#             fname = notcars[w - offset]
#             copyfile(fname, out_path+'/hardneg_'+str(w - offset)+'.png')
#         elif not neg and y_pred[w] == 0 and y[w] == 1:
#             fname = cars[w]
#             copyfile(fname, out_path+'/hardpos_'+str(w)+'.png')
# make_hardneg('data/hardpos/', False)

# Print Confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X)
conf = confusion_matrix(y, y_pred)
print(conf)

[[9816   47]
 [  41 8751]]


In [13]:
from sklearn.externals import joblib
config = dict(color_space=color_space, 
            spatial_size=spatial_size, hist_bins=hist_bins, 
            orient=orient, pix_per_cell=pix_per_cell, 
            cell_per_block=cell_per_block, 
            hog_channel=hog_channel, spatial_feat=spatial_feat, 
            hist_feat=hist_feat, hog_feat=hog_feat)
joblib.dump({'model':clf, 'config':config}, 'models/clf_9764.pkl')

['models/clf_9764.pkl']